In [163]:
import numpy as np
import pandas as pd
from datetime import datetime

In [69]:
import yfinance as yf

In [230]:
djia_lst = ['MMM', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT']
#Get the monthly adjusted closing price of the constituents of Dow Jones Industrial Average (DJIA)
djia_s = yf.download(djia_lst, 
                      start='2017-12-01', 
                      end='2022-12-31', 
                      progress=False,
                      interval="1mo",
).drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)


In [231]:
#Get the annual total dividend of the last year
# Define the start and end dates (full year of 2022)
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)

# Download the dividends for each symbol and concatenate the results
dfs = []
for symbol in sorted(djia_lst):
    stock = yf.Ticker(symbol)
    dividends = stock.history(start=start_date, end=end_date)["Dividends"].to_frame(name=symbol)
    dfs.append(dividends)

#drop columns with NaN and/or zero values 
df = pd.concat(dfs, axis=1).dropna(axis=1, how='all')
#df = df.loc[:, (df != 0).any(axis=0)]
df.index.name = "Date"

#add 'Annual Dividend' as the last row 

df.loc['Annual Dividend'] = df.sum(axis=0)

#create a new frame with with 'Annual Dividend' row only

annual_div = df.loc['Annual Dividend'].to_frame()
div = annual_div.transpose()

In [232]:
#Compute the dividend yield of the last year
djia_close = yf.download(djia_lst, 
                      start='2021-12-01', 
                      end='2021-12-31', 
                      progress=False,
                      interval="1mo",
).drop(columns = ['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
print(djia_close)
dyieldnp = div.to_numpy()/djia_close.to_numpy()
print(dyield)
    

                 Close                                                  \
                  AAPL        AMGN         AXP          BA         CAT   
Date                                                                     
2021-12-01  177.570007  224.970001  163.600006  201.320007  206.740005   

                                                                      ...  \
                   CRM       CSCO         CVX         DIS        DOW  ...   
Date                                                                  ...   
2021-12-01  254.130005  63.369999  117.349998  154.889999  56.720001  ...   

                                                                       \
                  MRK        MSFT         NKE          PG         TRV   
Date                                                                    
2021-12-01  76.639999  336.320007  166.669998  163.580002  156.429993   

                                                                  
                   UNH           V

In [237]:
#Compute the continuously compounding return (difference in logarithms) of the DJIA stocks
ldjia_s = np.log(djia_s)
djia_r = ldjia_s.diff() 
print(djia_r)

#Find the mean and sample standard deviation (SD) of the returns of the stocks
mean = djia_r.mean(axis=0)*12
sd = djia_r.std(axis=0)*np.sqrt(12)
print(mean)
print(sd)

#Compute the sample variance-covariance matrix of the returns of the stocks
cov = djia_r.cov()
print(cov)

#Then proceed to Microsoft Excel to compute the portfolio's monthly variance (and hence SD) by setting the weight of each stock,
#and by using the above covariance matrix.
#Then compute the annual SD by multiplying the monthly SD by square root of 12.
#Then compute the portfolio mean return.
#After that, compute the Sharpe ratio of the portfolio.
#Lastly, use "Solver" to solve for the weight of each stock such that the Sharpe ratio is maximised.



           Adj Close                                                    \
                AAPL      AMGN       AXP        BA       CAT       CRM   
Date                                                                     
2017-12-01       NaN       NaN       NaN       NaN       NaN       NaN   
2018-01-01 -0.010693  0.067535  0.000906  0.183671  0.032467  0.108183   
2018-02-01  0.061892 -0.012331 -0.015675  0.021883 -0.046734  0.020334   
2018-03-01 -0.055735 -0.067536 -0.044350 -0.094652 -0.048021  0.000430   
2018-04-01 -0.015133  0.023192  0.056986  0.017175 -0.020704  0.039535   
...              ...       ...       ...       ...       ...       ...   
2022-08-01 -0.033093 -0.029401 -0.013202  0.005883 -0.063801 -0.164420   
2022-09-01 -0.127556 -0.056319 -0.119273 -0.280284 -0.118433 -0.081923   
2022-10-01  0.103956  0.181841  0.095640  0.162961  0.277052  0.122530   
2022-11-01 -0.035243  0.057672  0.063559  0.227303  0.094831 -0.014497   
2022-12-01 -0.128762 -0.079741 -0.0644

In [242]:
companies=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
table  = companies[0]
splst = table
print(splst['Symbol'])

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     YUM
499    ZBRA
500     ZBH
501    ZION
502     ZTS
Name: Symbol, Length: 503, dtype: object


In [243]:
with pd.ExcelWriter('DJIA_2023.xlsx') as writer:  
    djia_s.to_excel(writer, sheet_name='s')
    djia_r.to_excel(writer, sheet_name='r')
    cov.to_excel(writer, sheet_name='cov')
    mean.to_excel(writer, sheet_name='miu')
    sd.to_excel(writer, sheet_name='sd')
    splst['Symbol'].to_excel(writer, sheet_name='splst')